In [1]:
# load data

import torch
import copy
data_name='topk'
file_path = 'data/citeseer_fixed_sbert.pt'
data_org = torch.load(file_path, map_location='cpu')# data for test

# org data for test
#print(data_org.y[14])
print(data_org.label_names)
# data for train
data_train = copy.deepcopy(data_org)  
#print(data_train.label_names)
all_number=3186



/tmp/ipykernel_781528/528042771.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_org = torch.load(file_path, map_location='cpu')# data for test


['Agents', 'ML', 'IR', 'DB', 'HCI', 'AI']


/home/chunjielu/miniconda3/envs/faiss_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# randomly set train& test dataset on GNN

def random_split_masks(data, num_train=120, num_val=500, num_test=1000, seed=42):

    """
    randomly set the data for train, val, test with certain number
    """
    torch.manual_seed(seed)  

    num_nodes = data.num_nodes
    all_indices = torch.randperm(num_nodes)  

    # index rearrange
    train_idx = all_indices[:num_train]
    val_idx = all_indices[num_train:num_train + num_val]
    test_idx = all_indices[num_train + num_val:num_train + num_val + num_test]


    data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    data.train_mask[train_idx] = True
    data.val_mask[val_idx] = True
    data.test_mask[test_idx] = True

    return data 

data_org = random_split_masks(data_org)
data_train = random_split_masks(data_train)

"""
print("Train size:", data_train.train_mask.sum().item())
print("Val size:", data_train.val_mask.sum().item())
print("Test size:", data_train.test_mask.sum().item())
print("是否有交集:", (data_train.train_mask & data_train.val_mask).any().item() or 
                   (data_train.train_mask & data_train.test_mask).any().item() or 
                   (data_train.val_mask & data_train.test_mask).any().item())
"""

'\nprint("Train size:", data_train.train_mask.sum().item())\nprint("Val size:", data_train.val_mask.sum().item())\nprint("Test size:", data_train.test_mask.sum().item())\nprint("是否有交集:", (data_train.train_mask & data_train.val_mask).any().item() or \n                   (data_train.train_mask & data_train.test_mask).any().item() or \n                   (data_train.val_mask & data_train.test_mask).any().item())\n'

In [4]:
import torch

# load data
file_path = 'LLM_GNN_data/citeseer_openai.pt'  
data = torch.load(file_path, map_location='cpu')


consistency_data = data[data_name]
class_map = { # a mapping from the responses
    'Agents': 0,
    'machine learning': 1,
    "information retrieval": 2,
    'database': 3,
    'human computer interaction': 4,
    "artificial intelligence": 5

}
#print(consistency_data)


/tmp/ipykernel_781528/3637104084.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path, map_location='cpu')


In [5]:

# process the responses (string)
unknown_labels = set()

def get_result(s):
    #print(s)
    """
    return the class [0,7) and it's confidence score from the string returned by LLM
    """
    l=0
    while(s[l]!=":"):
        l+=1
    l+=2
    if s[l-1]!='"' and s[l-1]!="'":
        l+=1

    r=l
    #print(r)
    while(s[r]!='"' and s[r]!="'"):
        r+=1
    label_str=s[l:r]
    #print(label_str)
    conf=0
    for i in s:
        if ord(i)>=ord('0') and ord(i)<=ord('9'):
            conf=conf*10+ord(i)-ord('0')

    if label_str not in class_map:
        unknown_labels.add(label_str)
    
    else:
        return class_map[label_str],conf

    


result=[] # (index, (label, confidecnce score))
for i, item in enumerate(consistency_data):
    if not item or not isinstance(item, list) or not any(s.strip() for s in item):
        continue
    if get_result(item[0])!=None:
        result.append([i,get_result(item[0])])
    #print(i,get_result(item[0]))

#print(result)
total=0
op=0

mask_list=[]
label_list=[]
anao=[0 for i in range(len(consistency_data))] 

for i in range(len(result)):
    idx=result[i][0]
    anao[idx]=result[i][1][1]/100
    label=result[i][1][0]
    mask_list.append(idx)
    label_list.append(label)
    if data_org.y[idx].item()==label:
        op+=1
    total+=1
print(f'accuracy of all the annotations: {op/total}') # the accuracy of response compared with the ground truth

annotations=torch.tensor(anao)
mask_list_tensor=torch.tensor(mask_list)
label_list_tensor=torch.tensor(label_list)
print(unknown_labels)


accuracy of all the annotations: 0.609504132231405
{'computer vision', 'software development', 'Multi-Agent Systems', 'control', 'agents'}


In [6]:
# set the training mask and label into the responses'data
def change_trainmask_and_label(mask_list,label_list,data):
    train_mask = torch.zeros(data_train.num_nodes, dtype=torch.bool)
    train_mask[mask_list] = True
    data.train_mask = train_mask
    data.y[data_train.train_mask] = label_list

change_trainmask_and_label(mask_list_tensor,label_list_tensor,data_train)

train_nodes = data_train.train_mask.sum().item() 


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
class GCN2(torch.nn.Module):
    def __init__(self, num_layers, input_dim, hidden_dimension, num_classes, dropout, norm=None) -> None:
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.norms = torch.nn.ModuleList()
        self.num_layers = num_layers
        self.dropout = dropout
        if num_layers == 1:
            self.convs.append(GCNConv(input_dim, num_classes, cached=False,
                             normalize=True))
        else:
            self.convs.append(GCNConv(input_dim, hidden_dimension, cached=False,
                             normalize=True))
            if norm:
                self.norms.append(torch.nn.BatchNorm1d(hidden_dimension))
            else:
                self.norms.append(torch.nn.Identity())

            for _ in range(num_layers - 2):
                self.convs.append(GCNConv(hidden_dimension, hidden_dimension, cached=False,
                             normalize=True))
                if norm:
                    self.norms.append(torch.nn.BatchNorm1d(hidden_dimension))
                else:
                    self.norms.append(torch.nn.Identity())

            self.convs.append(GCNConv(hidden_dimension, num_classes, cached=False, normalize=True))

    def forward(self, data):
        x, edge_index, edge_weight= data.x, data.edge_index, data.edge_weight
        for i in range(self.num_layers):
            x = F.dropout(x, p=self.dropout, training=self.training)
            if edge_weight != None:
                x = self.convs[i](x, edge_index, edge_weight)
            else:
                x = self.convs[i](x, edge_index)
            if i != self.num_layers - 1:
                x = self.norms[i](x)
                x = F.relu(x)
        return x

model = GCN2(num_layers=2, input_dim=data_train.num_node_features, hidden_dimension=64, 
            num_classes=6, dropout=0.5, norm=True)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

# 训练函数
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# 测试函数
def test(data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        accs.append(correct / mask.sum().item())
    return accs  # train_acc, val_acc, test_acc
def test_final(data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.test_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        accs.append(correct / mask.sum().item())
    return accs  # train_acc, val_acc, test_acc

# 训练流程
for epoch in range(1, 201):
    loss = train(data_train)
    train_acc, val_acc = test(data_train)
    if epoch % 20 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}")

Epoch 001, Loss: 2.3848, Train: 0.2169, Val: 0.2220
Epoch 020, Loss: 1.0587, Train: 0.8368, Val: 0.5640
Epoch 040, Loss: 0.6727, Train: 0.8843, Val: 0.5800
Epoch 060, Loss: 0.5656, Train: 0.9091, Val: 0.5860
Epoch 080, Loss: 0.4771, Train: 0.9339, Val: 0.5960
Epoch 100, Loss: 0.4263, Train: 0.9442, Val: 0.5940
Epoch 120, Loss: 0.3310, Train: 0.9649, Val: 0.6000
Epoch 140, Loss: 0.3546, Train: 0.9752, Val: 0.6040
Epoch 160, Loss: 0.3297, Train: 0.9814, Val: 0.6040
Epoch 180, Loss: 0.2734, Train: 0.9855, Val: 0.5980
Epoch 200, Loss: 0.2959, Train: 0.9938, Val: 0.6040


In [8]:
print(f'Number of training nodes: {train_nodes}')
print(f'final accuracy: {test_final(data_org)}')

Number of training nodes: 484
final accuracy: [0.59]
